# Cleaning data

This nb focuses on cleaning and filtering data.

In [1]:
import numpy as np
import pandas as pd
from datetime import date
import requests
import json

#### Handy functions

In [2]:
def USD_converter(to_cur, amount=1):
    """
    based by default from USD
    returns USD from COP
    """
 
    url = "https://api.exchangerate-api.com/v4/latest/USD"

    response = requests.get(url)
    data = response.text
    parsed = json.loads(data)
    date = parsed["date"]
    rates = parsed["rates"]
    print("Date:", date, "\n", "1 USD to COP: ", rates[to_cur], "\n", str(amount) + "COP = " 
          + str(amount / rates[to_cur]) + " USD" ) 
    return [rates[to_cur], amount / rates[to_cur]] 

###############################################

def km_latlon(lat1, lon1, lat2, lon2):
    """
    returns distance between two coordinates (latitude, longitude)
    An approach to determine location from properties
    """
    lat1, lat2 = np.radians(lat1), np.radians(lat2)
    lon1, lon2 = np.radians(lon1), np.radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (np.sin(dlat / 2)) ** 2 + np.cos(lat1) * np.cos(lat2) * (np.sin(dlon / 2)) ** 2
    c = 2 * np.arctan2( np.sqrt(a), np.sqrt(1-a) )
    return  6373 * c 

## Uploading data

In [3]:
### Properties from db Company A
properties_df = pd.read_csv('co_properties.csv.gz', compression='gzip', 
                                 header=0, sep=',', quotechar='"')
### bogota_locations df
bogota_df = pd.read_csv('bogota_localidad.csv', sep=';')
bogota_df = bogota_df.drop([5], axis=0) #Bogota row no needed
bogota_df = bogota_df[['LOCALIDAD', 'LONGITUD', 'LATITUD']]
bogota_df['ESTRATO_MEDIO'] = [4.3, 2.2, 3.4, 3.5, 3.9, 1.6, 1.6, 2.3, 2.7, 3.6, 3.2, 1.6, 3.6, 2.3, 1.8, 3.1, 3.2, 3.4, 1.8, 1.5]
bogota_df['SUPERFICIE [km2]'] = [38.9, 9.9, 4.8, 17.3, 65.3, 23.9, 130.0, 13.8, 38.6, 11.9, 35.9, 780.0, 14.2, 2.0, 45.2, 100.6, 33.3, 6.5, 49.1, 215.1]
bogota_df = bogota_df.round({'LONGITUD': 3, 'LATITUD': 3})

### amsl locations Bogota 
amsl_df = pd.read_csv('altura-bogota-localidad.csv', sep=",")

## Cleaning and filtering data

In [4]:
## Filtering 
sel_property = properties_df[['start_date', 'lat', 'lon', 'l3','surface_total' ,'price', 
                              'currency', 'property_type', 'operation_type']]
sel_property = sel_property.loc[(sel_property['l3'] == 'Bogotá D.C') 
                                & (sel_property['operation_type'] == 'Venta') 
                                & (sel_property['property_type'] == 'Apartamento')
                                & (sel_property['start_date'] > '2020-01-01') 
                                & (sel_property['start_date'] <= '2020-01-30')]
#Cleaning
sel_property = sel_property.dropna()

In [5]:
### Obtaining loc with mean class(estrato), respectively. Loc is a dict (cointaing index and loc)
###

loc = {}
strat = []

for i in range(len(sel_property)):
    diff = {}
    lat1 = sel_property['lat'].iloc[i]
    lon1 = sel_property['lon'].iloc[i]
    for n, j in enumerate(bogota_df['LOCALIDAD']):
        lat2 = bogota_df['LATITUD'].iloc[n]
        lon2 = bogota_df['LONGITUD'].iloc[n]
        distance = km_latlon(lat1, lon1, lat2, lon2)
        if distance <= bogota_df['SUPERFICIE [km2]'].iloc[n]: 
            #loc.append(j)
            diff[j] = distance
            #print(i, distance, j)
    if diff:
        #print(i, min(diff, key=diff.get))
        strat.append(bogota_df.loc[bogota_df['LOCALIDAD'] == min(diff, key=diff.get), 'ESTRATO_MEDIO'].item())
        loc[i] = min(diff, key=diff.get)

#### Final properties dataset 

In [6]:
data_companyA = sel_property
data_companyA['index'] = [i for i in range(len(sel_property))]
data_companyA['location'] = data_companyA['index'].map(loc)
data_companyA['mean_class'] = strat
data_companyA = data_companyA.drop(columns=['index', 'l3', 'start_date', 'property_type', 'operation_type'])

### Dataset USD per square meter ($USD / m^2$)

In [8]:
df = data_companyA

In [17]:
#df = data_companyA
## Price per m2
df['price_per_m2'] = df['price'] / df['surface_total']

## df_COL
df_COP = df[['location', 'mean_class', 'surface_total', 'price', 'price_per_m2', 'currency', 'lat', 'lon']].copy()

## df_USD
df_USD = df_COP[['lat', 'lon','location', 'mean_class', 'surface_total', 'price', 'price_per_m2', 'currency']]
df_USD['price'] =  df_USD['price'] / USD_converter('COP')[0]
df_USD['price_per_m2'] =  df_USD['price_per_m2'] / USD_converter('COP')[0]
df_USD['currency'] = 'USD'
df_USD = df_USD.round({'price':2, 'price_per_m2':2})

Date: 2021-01-13 
 1 USD to COP:  3457.564191 
 1COP = 0.00028922094999797506 USD
Date: 2021-01-13 
 1 USD to COP:  3457.564191 
 1COP = 0.00028922094999797506 USD


#### Average $USD / m^2$

In [18]:
df_USD['avg_price'] = df_USD.groupby('location')['price'].transform(np.mean)
df_USD['avg_price_m2'] = df_USD.groupby('location')['price_per_m2'].transform(np.mean)
df_USD = df_USD.round({'avg_price':2, 'avg_price_m2':2})

#### amsl dataset

In [20]:
amsl_df["LocNombre"].replace({"ANTONIO NARIÃ‘O": "ANTONIO NARIÑO", "CANDELARIA": "LA CANDELARIA", 
                              "USAQUEN":"USAQUÉN", "CIUDAD BOLIVAR":"CIUDAD BOLÍVAR", 
                              "ENGATIVA": "ENGATIVÁ", "FONTIBON":"FONTIBÓN", 
                              "LOS MARTIRES":"LOS MÁRTIRES"}, inplace=True)

## Save datasets `.csv`

In [21]:
data_companyA.to_csv('data_companyA.csv', index=False)
amsl_df.to_csv('amsl_bogotá_csv.csv', index=False)
df_USD.to_csv('data_USD.csv', index=False)

**Some useful resources**

[**Movilidad Bogotá, Caracterización Socioeconomica de Bogotá**](https://www.movilidadbogota.gov.co/web/sites/default/files/Paginas/28-04-2020/06-caracterizacion_socioeconomica_de_bogota_y_la_region_-_v8.pdf)

[**Datos Localidades de Bogotá SDP**](http://www.sdp.gov.co/sites/default/files/caracterizacion_sisben-04032015.pdf)